In [ ]:
import threading 
import sys

### Using multi-threading, implement a concurrent program with two threads: the first thread prints even numbers lower than N, the second thread prints odd numbers lower than N

- What happens when N increases:  10, 100, 1000..?
- Add a countdown in the main thread (from N to 0): how do the 3 threads synchronize?
- Can we force the execution order: Even, Odd, Countdown ?

In [ ]:
N = 1000
def thread_function(name,remainder,tojoin=None):
    #if tojoin:
    #    tojoin.join()
    for i in range(N):
        if i%2==remainder:
            print(f"Thread {name}: {i}\n", end='')
            
            
te = threading.Thread(target=thread_function, args=("Even", 0))
to = threading.Thread(target=thread_function, args=("Odds", 1,te))

te.start()
to.start()

#to.join()

for i in range(N,0, -1):
    print(f"Count Down: {i}\n", end='')

### Create a multithreaded counter (4 threads). Each thread increments the counter by 1 for 1,000,000 times.
- What is the main problem?
- How to force the program to get to the correct result? If you force the program to get the correct result, does it take longer to execute? If so, why?

In [ ]:
counter = 0

#lock = threading.Lock()

def increase_counter():
    global counter
    for i in range(1000000):
        #lock.acquire()
        counter = counter + 1
        #lock.release()

t1 = threading.Thread(target=increase_counter)
t2 = threading.Thread(target=increase_counter)
t3 = threading.Thread(target=increase_counter)
t4 = threading.Thread(target=increase_counter)

t1.start()
t2.start()
t3.start()
t4.start()
t1.join()
t2.join()
t3.join()
t4.join()
print(counter)

# With locks it takes longer, because only one thread at a time can use the counter, 
# the other three threads have to wait!

### Create a consumer/producer architecture, where both the consumer and the producer are run by threads

Task: check the primality of numbers.
The consumer adds tasks to a list (in form of the numbers to be checked) and the consumer performs these tasks (check the primality of the number). The list should be managed by a properly locked data structure.

*Caveat: in Python you cannot kill threads, thus you may need to restart the Jupyter notebook's kernel to stop the execution ...*


In [ ]:
import random
import time

# Here, we define a new class for the task list; there are other possibilities
class Storage:
    def __init__(self):
        self.list_tasks = []
        self.lock = threading.Lock()
    
    def get_task(self):
        self.lock.acquire()
        t = None
        if len(self.list_tasks) > 0:
            t = self.list_tasks[0]
            self.list_tasks = self.list_tasks[1:]
        self.lock.release()
        return t
    
    def add_task(self, t):
        self.lock.acquire()
        self.list_tasks.append(t)
        print(f"Added task {t}")
        self.lock.release()
        
storage = Storage()
        
def producer():
    while True:
        time.sleep(random.random())
        storage.add_task(random.choice(range(10000)))

def is_prime(n):
    for i in range(2, int(n**.5 +1)):
        if n%i == 0:
            return False
    return True

def consumer():
    while True:
        time.sleep(random.random())
        t = storage.get_task()
        if t:
            print(f"The number {t} is {'' if is_prime(t) else 'not '}prime\n", end='')
            
t_prod = threading.Thread(target=producer)
t_cons = threading.Thread(target=consumer)

t_prod.start()
t_cons.start()

### Multiprocess Pool
Create a program that generates a matrix of 200 rows and 50k columns of random numbers between 0 and 100k. Then compute the total sum of all squared values sequentially and in parallel. For parallel exectuion use a Pool of threads.

In [ ]:
import multiprocessing as mp
import random

matrix = []
for i in range(200):
    matrix.append(random.sample(range(100000), 50000))

def sum_square(l):
    total = 0
    for e in l:
        total+=e**2
    return total

starttime=time.time()
print(sum(map(sum_square, matrix)))
print(f"sequential time: {time.time() - starttime}")

starttime=time.time()
with mp.Pool(4) as p:
    print(sum(p.map(sum_square, matrix)))
print(f"parallel time: {time.time() - starttime}")